In [1]:
!pip install pandas pyarrow scikit-learn gdown
!gdown https://drive.google.com/uc?id=1HjAHsB2zpUxA9Do3tBwjPbVQHEGFfEaU
!gdown https://drive.google.com/uc?id=1nrOugQVzM5T0GbBwwd0yf3t4d5ijegDQ
!gdown https://drive.google.com/uc?id=1SfBZqoUZs-nqZocLdaF7-1IPORIF1a_s
!gdown https://drive.google.com/uc?id=1wpfkK6AEPolTNI2Q-BO6wLop6qsdk4Zu

     --------------------------------------- 20.6/20.6 MB 10.6 MB/s eta 0:00:00


Downloading...
From (uriginal): https://drive.google.com/uc?id=1HjAHsB2zpUxA9Do3tBwjPbVQHEGFfEaU
From (redirected): https://drive.google.com/uc?id=1HjAHsB2zpUxA9Do3tBwjPbVQHEGFfEaU&confirm=t&uuid=3f5e7be5-3484-4a0d-b9a9-94c26181f7de
To: C:\Users\Максим\Alfa\dataset_train_small.parquet

  0%|          | 0.00/396M [00:00<?, ?B/s]
  0%|          | 524k/396M [00:00<01:22, 4.79MB/s]
  1%|          | 2.10M/396M [00:00<00:40, 9.63MB/s]
  1%|          | 3.15M/396M [00:00<00:48, 8.09MB/s]
  1%|1         | 5.24M/396M [00:00<00:33, 11.7MB/s]
  2%|1         | 6.82M/396M [00:00<00:32, 12.0MB/s]
  2%|2         | 8.39M/396M [00:00<00:34, 11.3MB/s]
  3%|2         | 9.96M/396M [00:00<00:33, 11.7MB/s]
  3%|2         | 11.5M/396M [00:01<00:38, 9.91MB/s]
  4%|3         | 14.2M/396M [00:01<00:30, 12.4MB/s]
  4%|3         | 15.7M/396M [00:01<00:30, 12.5MB/s]
  4%|4         | 17.3M/396M [00:01<00:30, 12.5MB/s]
  5%|4         | 18.9M/396M [00:01<00:31, 12.1MB/s]
  5%|5         | 20.4M/396M [00:01<00:30, 12.2M

In [152]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [3]:
dataset_train = pd.read_parquet("dataset_train_small.parquet")
target_train = pd.read_parquet("target_train_small.parquet")

In [ ]:
dataset_train

,app_id,amnt,currency,operation_kind,card_type,operation_type,operation_type_group,ecommerce_flag,payment_system,income_flag,mcc,country,city,mcc_category,day_of_week,hour,days_before,weekofyear,hour_diff,transaction_number
0,92990,0.301232,1,1,96,2,1,1,3,1,8,1,49,1,7,15,92,42,22,1
1,92990,0.329823,1,1,96,2,1,1,3,1,1,1,49,1,7,15,92,42,0,2
2,92990,0.294431,1,1,96,2,1,1,3,1,1,1,49,1,5,11,90,48,44,3
3,92990,0.302912,1,1,96,2,1,1,3,1,1,1,49,1,4,15,89,48,28,4
4,92990,0.359050,1,2,102,8,1,1,3,1,2,1,49,2,3,11,88,48,20,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32635982,92988,0.361067,1,1,54,2,1,1,3,1,1,1,2,1,7,7,1,2,21,39
32635983,92988,0.356763,1,1,54,2,1,1,3,1,1,1,2,1,7,7,1,2,0,40
32635984,92989,0.218385,1,1,52,2,1,1,3,1,4,1,2,3,3,9,81,46,480,1
32635985,92989,0.488173,1,4,52,4,2,1,3,2,2,1,15,2,4,10,75,20,144,2


In [4]:
from nltk import WordNetLemmatizer
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

In [61]:
from sklearn.ensemble import RandomForestClassifier

In [5]:
target_train

,app_id,product,flag
0,1,1,0
1,5,1,0
2,6,1,0
3,7,0,0
4,11,1,0
...,...,...,...
377341,805124,0,0
377342,805125,0,0
377343,805127,4,0
377344,805130,1,0


In [6]:
def num_unique_values(x):
    return len(set(x))

In [7]:
def get_features(dataset, target):
    features = dataset.groupby("app_id", as_index=False).agg({
        "transaction_number": "max",
        "amnt": ["min", "max", "mean"],
        "currency": num_unique_values
    })
    features.columns = ['_'.join(col).strip('_') for col in features.columns.values]
    features = features.join(target.set_index("app_id"), "app_id")
    return features.drop(columns=["app_id", "flag"]), features[["app_id", "flag"]]

In [8]:
X_train, y_train = get_features(dataset_train, target_train)

In [9]:
from sklearn import tree

In [10]:
X_train

,transaction_number_max,amnt_min,amnt_max,amnt_mean,currency_num_unique_values,product
0,120,0.155581,0.539584,0.315662,1,1
1,47,0.269110,0.594620,0.413886,1,1
2,23,0.321124,0.605813,0.405220,3,1
3,28,0.062575,0.476811,0.248939,1,0
4,17,0.258972,0.516834,0.395651,1,1
...,...,...,...,...,...,...
377341,20,0.178216,0.454634,0.327272,1,0
377342,92,0.212853,0.607141,0.341180,1,0
377343,3,0.520103,0.520103,0.520103,1,4
377344,53,0.271747,0.563568,0.406822,3,1


In [11]:
y_train

,app_id,flag
0,1,0
1,5,0
2,6,0
3,7,0
4,11,0
...,...,...
377341,805124,0
377342,805125,0
377343,805127,0
377344,805130,0


In [224]:
model = RandomForestClassifier(n_estimators=200,
                               bootstrap = True,
                               max_features = 'sqrt' ,max_depth=10,min_samples_split=5, random_state=1,criterion="log_loss" )
model.fit(X_train, y_train["flag"])
y_train_pred = model.predict_proba(X_train)[:, 1]
print(200 * roc_auc_score(y_train["flag"], y_train_pred) - 100)

55.32173230608862


In [225]:
dataset_test = pd.read_parquet("dataset_test.parquet")
target_test = pd.read_parquet("target_test_contest.parquet")

In [226]:
target_test

,app_id,product
0,805133,1
1,805134,0
2,805135,1
3,805136,1
4,805137,2
...,...,...
188669,1003045,1
188670,1003047,0
188671,1003048,1
188672,1003049,0


In [227]:
target_test["flag"] = 0

In [228]:
X_test, y_test = get_features(dataset_test, target_test)

In [229]:
X_test

,transaction_number_max,amnt_min,amnt_max,amnt_mean,currency_num_unique_values,product
0,16,0.271738,0.562335,0.364273,1,1
1,11,0.290811,0.657194,0.424697,1,0
2,242,0.000000,0.646036,0.387859,1,1
3,33,0.000000,0.560044,0.344616,1,1
4,23,0.196203,0.582099,0.358440,1,2
...,...,...,...,...,...,...
188669,8,0.423667,0.507530,0.450073,1,1
188670,52,0.000000,0.652660,0.389890,1,0
188671,112,0.225878,0.683797,0.404899,2,1
188672,6,0.345373,0.510295,0.478856,1,0


In [230]:
y_test_pred = model.predict_proba(X_test)[:, 1]
y_test["flag"] = y_test_pred

In [231]:
y_test

,app_id,flag
0,805133,0.020261
1,805134,0.016563
2,805135,0.029987
3,805136,0.030402
4,805137,0.052398
...,...,...
188669,1003045,0.020630
188670,1003047,0.023698
188671,1003048,0.013831
188672,1003049,0.018457


In [232]:
X_test

,transaction_number_max,amnt_min,amnt_max,amnt_mean,currency_num_unique_values,product
0,16,0.271738,0.562335,0.364273,1,1
1,11,0.290811,0.657194,0.424697,1,0
2,242,0.000000,0.646036,0.387859,1,1
3,33,0.000000,0.560044,0.344616,1,1
4,23,0.196203,0.582099,0.358440,1,2
...,...,...,...,...,...,...
188669,8,0.423667,0.507530,0.450073,1,1
188670,52,0.000000,0.652660,0.389890,1,0
188671,112,0.225878,0.683797,0.404899,2,1
188672,6,0.345373,0.510295,0.478856,1,0


In [233]:
y_test.to_csv("result.csv", index=False)